# GCS Auth

In [ ]:
print("Setting up GCS access...")
import os
os.environ['USE_AUTH_EPHEM'] = '0'
from google.colab import auth
auth.authenticate_user()

# Set Up

In [ ]:
from IPython.display import clear_output 
!pip install gcsfs
!pip install t5==0.9.2
!pip install -q tensorflow-text==2.8.0rc0
clear_output()
print("Installing dependencies...")
import functools
import os
import gin
import tensorflow_gcs_config
from google.colab import auth
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
from contextlib import contextmanager
import logging as py_logging
import t5

In [ ]:
tf.app.flags.DEFINE_string ('f', '', '')
# Set credentials for GCS reading/writing from Colab and TPU.
TPU_TOPOLOGY = "2x2"
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  TPU_ADDRESS = tpu.get_master()
  print('Running on TPU:', TPU_ADDRESS)
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
tf.config.experimental_connect_to_host(TPU_ADDRESS)
tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

#LOGGING
tf.get_logger().propagate = False
py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

# Variables and constants


In [5]:
representation = "tokens" #@param ['tokens']
pretraining_task = "masking" #@param ['masking']
scheduler = "isr" #@param ["polynomial", "constant", "isr", "slanted"]
my_finetuning_task = "single-log-injection" #@param ['multi-log-injection', 'single-log-injection', 'classifier']
multi_log_injection_type = "one-to-n" #@param [ "one-to-n"]
prefix = 'SINGLE_LOG_INJECTION: ' #@param ['SINGLE_LOG_INJECTION: ', 'CLASSIFICATION: ', 'MULTI_LOG_INJECTION: ']
TASK_NAME = "log_injection" #@param ['classification', 'log_injection']

# Load Vocabulary

In [6]:
vocab_model_path = f"gs://lance2/tokenizer/{representation}/tokenizer.model"
vocab_path = f"gs://lance2/tokenizer/{representation}/tokenizer.vocab"

In [7]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary

num_special_mask_tokens = 100 #@param {type: "integer"}

def load_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, num_special_mask_tokens)

# Prepare Dataset for T5

In [ ]:
if my_finetuning_task == "multi-log-injection":
  train_path = f'gs://lance2/finetune/{my_finetuning_task}/{multi_log_injection_type}/{representation}/train.tsv'
  eval_path = f'gs://lance2/finetune/{my_finetuning_task}/{multi_log_injection_type}/{representation}/eval.tsv'
  test_path = f'gs://lance2/finetune/{my_finetuning_task}/{multi_log_injection_type}/{representation}/test.tsv'
else:
  train_path = f'gs://lance2/finetune/{my_finetuning_task}/{representation}/train.tsv'
  eval_path = f'gs://lance2/finetune/{my_finetuning_task}/{representation}/eval.tsv'
  test_path = f'gs://lance2/finetune/{my_finetuning_task}/{representation}/test.tsv'

finetune_datasets_paths = {
    "train":      train_path,
    "validation": eval_path,
    # "test": test_path
}
train_path

In [9]:
def load_dataset(split, shuffle_files=True):
  """
  Function to load .tsv dataset as a tf.data.Dataset in TensorFlow
  """
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(finetune_datasets_paths[split])
  ds = ds.map(functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                          field_delim="\t", use_quote_delim=False)
                          , 
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

### A few examples

In [ ]:
print("A few raw validation examples...")
for ex in tfds.as_numpy(load_dataset("validation").take(5)):
  print(ex)

# Dataset Prepocessing 

In [11]:
from tensorflow_datasets.core.utils.type_utils import Shape
def preprocessing(ds):
  """
  Preprocess function to convert the tf.data.Dataset into a text-to-text format,
  with both inputs and targets fields.
  Param: tf.data.Dataset
  Return: text-to-text format
  """
  def to_inputs_and_targets(ex):
    x_input = tf.strings.strip(prefix + ex['input'])
    y_label = tf.strings.strip(ex['output']) 
    inputs = tf.strings.join([x_input], separator=' ')
    class_label = tf.strings.join([y_label], separator=' ')
    return {'inputs': inputs, 'targets': class_label}
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

### A few examples

In [ ]:
print("A few preprocessed train examples...")
sample = tfds.as_numpy(preprocessing(load_dataset("train").take(5)))
for ex in sample:
  print(ex)

# Creating Task and Mixture

In [ ]:
DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=load_vocabulary(), add_eos=True, required=False),
    "targets": Feature(
        vocabulary=load_vocabulary(), add_eos=True)
    }

# TASK
t5.data.TaskRegistry.remove(TASK_NAME)
t5.data.TaskRegistry.add(
    TASK_NAME,
    # Function which returns a tf.data.Dataset
    dataset_fn=load_dataset,
    splits=["train",
            "validation",
            # "test"
            ],
    # List of functions that preprocess the input tf.data.Dataset
    text_preprocessor=[preprocessing],
    # Accuracy is used as evaluation metric
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, helps for mixing and auto-caching
    # num_input_examples=num_input_examples,
    output_features = DEFAULT_OUTPUT_FEATURES
)

MIXTURE_NAME = "finetuning" #@param{ type : "string"}

# MIXTURE
t5.data.MixtureRegistry.remove(MIXTURE_NAME)
t5.data.MixtureRegistry.add(
    MIXTURE_NAME,
    # List of tasks
    [TASK_NAME],
    default_rate=1.0
)

### A few examples

In [ ]:
finetuning_task = t5.data.TaskRegistry.get(TASK_NAME)
ds = finetuning_task.get_dataset(split="train", sequence_length={"inputs": 512, "targets": 512})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

# Creating Model

In [15]:
# Storage paths
if my_finetuning_task == "multi-log-injection":
  FINETUNE_MODEL_DIR = f"gs://lance2/finetuned-model/{my_finetuning_task}/{multi_log_injection_type}/{representation}/{scheduler}"
else:
  FINETUNE_MODEL_DIR = f"gs://lance2/finetuned-model/{my_finetuning_task}/{representation}/{scheduler}"
  
PRETRAIN_MODEL_DIR= f'gs://lance2/pretrained-model/{representation}/{pretraining_task}/'

In [16]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

# Learning rate properties
starter_learning_rate = 0.01 #@param {type : "number"}
end_learning_rate = 0.001 #@param {type : "number"}
decay_steps = 10000 #@param {type : "integer"}

learning_rate_fn = PolynomialDecay(
     starter_learning_rate,
     decay_steps,
     end_learning_rate,
     power=0.5)

In [ ]:
from mesh_tensorflow.transformer.learning_rate_schedules import slanted_triangular, truncated_rsqrt
from t5 import models

# Learning rate schedule fn
if scheduler == 'polynomial':
  learning_rate_scheduler = learning_rate_fn
elif scheduler == 'isr':
  learning_rate_scheduler = truncated_rsqrt
elif scheduler == 'slanted':
  learning_rate_scheduler = slanted_triangular
else: 
  learning_rate_scheduler = 0.001

print(learning_rate_scheduler)

# Model properties
MODEL_SIZE = "small" 
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 128, 100),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]


model = t5.models.MtfModel(
    model_dir=FINETUNE_MODEL_DIR,
    tpu=TPU_ADDRESS,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size, 
    sequence_length={"inputs": 1024, "targets": 1024},
    learning_rate_schedule = learning_rate_scheduler, 
    save_checkpoints_steps=10000,
    keep_checkpoint_max=keep_checkpoint_max
)

# Learning Scheduler

In [ ]:
BASE_DIR = f"gs://lance2/learning_rate_scheduler/{scheduler}"
print(BASE_DIR)
GIN_FILENAME = "operative_config.gin" #@param {type : "string"}
# 'no_pretraining_operative_config.gin'
remote_gin_path = os.path.join(BASE_DIR, GIN_FILENAME)
LOCAL_GIN_PATH = f"/content/{GIN_FILENAME}"
!gsutil -m cp $remote_gin_path $LOCAL_GIN_PATH

In [ ]:
with gin.unlock_config():
    gin.parse_config_file(LOCAL_GIN_PATH)

## Train

In [ ]:
TRAIN_STEPS =  100000 #@param {type: "integer"}

# Stat finetuning 
model.finetune(mixture_or_task_name=MIXTURE_NAME,
               finetune_steps=TRAIN_STEPS,
               pretrained_model_dir=PRETRAIN_MODEL_DIR)


# Evaluating the Model

In [ ]:
checkpoints = []
for check in range(500000, 600000, 10000):
  checkpoints.append(check)
# checkpoints

In [ ]:
from t5 import models

model = t5.models.MtfModel(FINETUNE_MODEL_DIR, tpu=TPU_ADDRESS)
# Evaluate on the validation sets of the tasks in our mixture
model.bach_size=16
model.eval(
    mixture_or_task_name=MIXTURE_NAME,
    checkpoint_steps=checkpoints,
    # checkpoint_steps=[600000],
    # split='test'
)